In [7]:
from pathlib import Path
from thor_scsi.factory import accelerator_from_config
from thor_scsi.pyflame import Config
import thor_scsi.lib as tslib

import numpy as np
import matplotlib.pyplot as plt

import gtpsa
import os
import time
import copy
import random

from multiprocessing import Process, Queue, Pipe
from queue import Empty

from scipy.stats import truncnorm, norm

In [2]:
def create_nlk_interpolation(nlk_name):
    def compute_mirror_position_plate(ref_pos, mirror_pos, *, y_plane=True):
        """ """
        assert y_plane
        dy = ref_pos.imag - mirror_pos.imag
        return ref_pos - 2 * dy * 1j

    # fmt: off
    ref_pos1 =  8e-3 +  7e-3j
    ref_pos2 = 17e-3 + 15e-3j
    # fmt: on
    t_current = -7e2

    # fmt: off
    t_current *= 1 - 1 * 0.14 / 2
    ref_pos1  *= 1 - 0.14
    ref_pos2  *= 1 - 0.14

    plate_position1 = 5e-3j
    mirror_pos1 = compute_mirror_position_plate(ref_pos1, plate_position1)

    inner = tslib.aircoil_filament(ref_pos1.real, ref_pos1.imag,  t_current)
    outer = tslib.aircoil_filament(ref_pos2.real, ref_pos2.imag, -t_current)
    mirror = tslib.aircoil_filament(mirror_pos1.real, mirror_pos1.imag, -t_current * 0.14)
    nlkf_intp = tslib.NonLinearKickerInterpolation([inner, outer, mirror])

    c = Config()
    c.setAny("L", 0e0)
    c.setAny("name", nlk_name)
    c.setAny("N", 1)
    nlk = tslib.FieldKick(c)
    nlk.set_field_interpolator(nlkf_intp)
    return nlk, nlkf_intp

In [3]:
emittance_start = 70e-9
nv = 6
mo = 1
default_desc = gtpsa.desc(nv, mo)


def calulate_sigma_px(sigma_x, *, emittance=emittance_start):
    sigma_px = np.sqrt(emittance ** 2 - sigma_x ** 2)
    return sigma_px

def create_state_space_vector(*, mu_x=0e0, mu_px=0e0, desc=default_desc):
    ps = gtpsa.ss_vect_double(desc, mo, nv)
    #ps.set_identity()
    ps.set_zero()
    ps[x_]+=mu_x
    ps[px_]+=mu_px
    return ps
    
    t_x = ps[x_]
    t_x.set(0, mu_x)
    t_px = ps[px_]
    t_px.set(0, mu_px)
    ps[x_] = t_x
    ps[px_] = t_px
    return ps

In [4]:
# t_file = Path(os.environ["HOME"]) / "Devel"/ "gitlab" / "dt4cc"/"lattices" / "b2_stduser_beamports_blm_tracy_corr.lat"
prefix = Path(os.environ["HOME"])
prefix = Path("/home/schnizer")
t_dir =  prefix / "Devel" / "gitlab" / "dt4acc" / "lattices"
t_file = t_dir / "b2_stduser_beamports_blm_tracy_corr_with_nlk.lat"

x_, px_ = 0, 1

    
def particle_propagation(index,acc_info):
    when_activate_NLK = 20000       #when to activate the NLK
    
    acc,calc_config,nlkfk,nlkf_intp = acc_info #get accelerator information
    
    mu_x = mu_x_list[index]        #mu_x,mu_px are global variables
    mu_px = mu_px_list[index]
    
    
    emittance_start = 70e-9
    

    mu_x_process = []      #save how the position changes during the rounds
    for runde in range(1000):
        if runde%10==0:      #due to memory problems we can't save every round
            mu_x_process.append(mu_x)
        
        if runde != when_activate_NLK:
            nlkf_intp.set_scale(0.0)
                   
        elif runde == when_activate_NLK:
            nlkf_intp.set_scale(.1)
        else:
            raise ValueError("should not end up here")
            
        ps = create_state_space_vector(mu_x=mu_x,mu_px=mu_px)
        
        result = acc.propagate(calc_config, ps)
        assert result==len(acc) 
        
        #update mu_x and mu_px
        n_mu_x = ps.cst()[x_]
        n_mu_px = ps.cst()[px_]
        mu_x=n_mu_x
        mu_px=n_mu_px

        if n_mu_x>0.015:    #septum splint
            return False, mu_x_process
        elif n_mu_x<=0.015:
            continue
        else:
            return False,mu_x_process   #n_mu_x might be NAN, if the settings are extreme

    return True,mu_x_process


In [5]:
#Create data

sigma_x = .25e-9
assert sigma_x <=emittance_start

mu_x_list = norm.rvs(loc=8e-3,scale=sigma_x,size=1000)
mu_px_list = norm.rvs(loc=2e-4,scale=calulate_sigma_px(sigma_x,emittance=emittance_start),size=1000)
print(mu_x_list[1],mu_x_list[2])


0.00800000017680404 0.007999999944724733


True

In [6]:






def run(idx, args, worker_acc_info):
    """
    Inputs:  -idx: Worker index
             -args: Job index
    """
    
    return particle_propagation(args, worker_acc_info)

    
    
def run_jobs(jobs, workers=1):
    
    q = Queue()  #Multiprocessing Queue to remember which jobs need to be done
    
    def worker(idx,q,send_end):
        """
        Each worker gets their individuall accelerator
        """
        
        acc = accelerator_from_config(t_file)
        calc_config = tslib.ConfigType()

        #Beschreibung von NLK
        nlkfk = acc.find("pkdnl1kr", 0)
        nlk_name = nlkfk.name
        _, nlkf_intp = create_nlk_interpolation(nlk_name)
        nlkfk.set_field_interpolator(nlkf_intp)
        assert(nlkfk.name == nlk_name)
        
        worker_acc_info = (acc,calc_config,nlkfk,nlkf_intp)   #compress all information
        
        
        try:
            while not q.empty():
                args = q.get(timeout=1)    #has form ("Job", job_index)
                print(args[1])
                
                for running_idx in range(25): #let each worker run multiple runs at once, 
                                              #to decrease time consumption
                        
                    result, process = run(idx,args[1]+running_idx, worker_acc_info)

                    send_end.send((idx,result, np.array(process)*1000))    #send information to output pipe
         
        except Empty:    #just for safety reasons
            return True

    for job in jobs:
        q.put(job)
    
    pipe_list=[]    #list of input/output pipes
    processes = []  #list of all workers
    
    for i in range(0, workers):
        recv_end, send_end = Pipe(False)
        pipe_list.append(recv_end)
        
        p = Process(target=worker, args=[i,q,send_end])
        p.daemon = True
        p.start()
        processes.append(p)

    for p in processes: 
        p.join()
    
    #need to return the pipe_list. analyzing it here gives errors
    return pipe_list

import time
if __name__ == "__main__":
    a=time.time()
    pipe_list = run_jobs([('job', i) for i in range(0,1000,25)], workers=20)
    print(time.time()-a)

0
25
50
75
100
125
200150
225
250
275
175
300

325
350
375

400425

450475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
66.37057447433472


In [ ]:
idx2 = 0         #running index
number_True = 0  #how many electrons made the 1000 circles
number_Total = 0
position_array = np.zeros((1000,100)).astype("float64")  #array to save movement of the electrons

for x in range(len(pipe_list)):

    for i in range(10000):
        #print(idx2)
        try:
            _,result,process = pipe_list[x].recv()

            number_Total+=1
            if result == True:
                number_True+=1

            position_array[idx2,:len(process)]=process
            idx2+=1
        except EOFError:
            #print(x,idx2)
            pipe_list[x].close()
            break
number_True,number_Total